In [1]:
import pandas as pd

athletes_df = pd.read_csv('/content/Athletes.csv')
coaches_df = pd.read_csv('/content/Coaches.csv')
entries_gender_df = pd.read_csv('/content/EntriesGender.csv')
medals_df = pd.read_csv('/content/Medals.csv')
teams_df = pd.read_csv('/content/Teams.csv')

print("Athletes DataFrame:")
display(athletes_df.head())

print("\nCoaches DataFrame:")
display(coaches_df.head())

print("\nEntriesGender DataFrame:")
display(entries_gender_df.head())

print("\nMedals DataFrame:")
display(medals_df.head())

print("\nTeams DataFrame:")
display(teams_df.head())

Athletes DataFrame:


,Name,NOC,Discipline
0,AALERUD Katrine,Norway,Cycling Road
1,ABAD Nestor,Spain,Artistic Gymnastics
2,ABAGNALE Giovanni,Italy,Rowing
3,ABALDE Alberto,Spain,Basketball
4,ABALDE Tamara,Spain,Basketball



Coaches DataFrame:


,Name,NOC,Discipline,Event
0,ABDELMAGID Wael,Egypt,Football,NaN
1,ABE Junya,Japan,Volleyball,NaN
2,ABE Katsuhiko,Japan,Basketball,NaN
3,ADAMA Cherif,Côte d'Ivoire,Football,NaN
4,AGEBA Yuya,Japan,Volleyball,NaN



EntriesGender DataFrame:


,Discipline,Female,Male,Total
0,3x3 Basketball,32,32,64
1,Archery,64,64,128
2,Artistic Gymnastics,98,98,196
3,Artistic Swimming,105,0,105
4,Athletics,969,1072,2041



Medals DataFrame:


,Rank,Team/NOC,Gold,Silver,Bronze,Total,Rank by Total
0,1,United States of America,39,41,33,113,1
1,2,People's Republic of China,38,32,18,88,2
2,3,Japan,27,14,17,58,5
3,4,Great Britain,22,21,22,65,4
4,5,ROC,20,28,23,71,3



Teams DataFrame:


,Name,Discipline,NOC,Event
0,Belgium,3x3 Basketball,Belgium,Men
1,China,3x3 Basketball,People's Republic of China,Men
2,China,3x3 Basketball,People's Republic of China,Women
3,France,3x3 Basketball,France,Women
4,Italy,3x3 Basketball,Italy,Women


**Step 1: Filter Athletes**: Filter the `athletes_df` to include only the countries (NOCs) that are present in the `medals_df`.

In [4]:
# Get the list of countries from the medals_df
medal_countries = medals_df['Team/NOC'].unique()

# Filter athletes_df to include only athletes from countries in medal_countries
filtered_athletes_df = athletes_df[athletes_df['NOC'].isin(medal_countries)]

display(filtered_athletes_df.head())

,Name,NOC,Discipline
0,AALERUD Katrine,Norway,Cycling Road
1,ABAD Nestor,Spain,Artistic Gymnastics
2,ABAGNALE Giovanni,Italy,Rowing
3,ABALDE Alberto,Spain,Basketball
4,ABALDE Tamara,Spain,Basketball


**Step 2: Count Athletes per Country**: Group the filtered `athletes_df` by NOC and count the number of athletes in each country.

In [6]:
# Group by NOC and count the number of athletes
athlete_counts = filtered_athletes_df['NOC'].value_counts().reset_index()
athlete_counts.columns = ['NOC', 'Number of Athletes']

display(athlete_counts.head())

,NOC,Number of Athletes
0,United States of America,615
1,Japan,586
2,Australia,470
3,People's Republic of China,401
4,Germany,400


**Step 3: Merge with Medals Data**: Merge the athlete counts into the `medals_df` based on the country (Team/NOC).

In [13]:
# Merge athlete_counts with medals_df
medals_with_athletes = pd.merge(medals_df, athlete_counts, left_on='Team/NOC', right_on='NOC', how='left')

# Drop the redundant 'NOC' column from the merge
medals_with_athletes = medals_with_athletes.drop('NOC', axis=1)

# Calculate the success ratio
medals_with_athletes['Success Ratio'] = medals_with_athletes['Total'] / medals_with_athletes['Number of Athletes'] * 100

display(medals_with_athletes.head(93))

,Rank,Team/NOC,Gold,Silver,Bronze,Total,Rank by Total,Number of Athletes,Success Ratio
0,1,United States of America,39,41,33,113,1,615,18.373984
1,2,People's Republic of China,38,32,18,88,2,401,21.945137
2,3,Japan,27,14,17,58,5,586,9.897611
3,4,Great Britain,22,21,22,65,4,366,17.759563
4,5,ROC,20,28,23,71,3,318,22.327044
...,...,...,...,...,...,...,...,...,...
88,86,Ghana,0,0,1,1,77,14,7.142857
89,86,Grenada,0,0,1,1,77,6,16.666667
90,86,Kuwait,0,0,1,1,77,10,10.000000
91,86,Republic of Moldova,0,0,1,1,77,19,5.263158


In [14]:
teams_df

,Name,Discipline,NOC,Event
0,Belgium,3x3 Basketball,Belgium,Men
1,China,3x3 Basketball,People's Republic of China,Men
2,China,3x3 Basketball,People's Republic of China,Women
3,France,3x3 Basketball,France,Women
4,Italy,3x3 Basketball,Italy,Women
...,...,...,...,...
738,South Africa,Water Polo,South Africa,Women
739,Spain,Water Polo,Spain,Men
740,Spain,Water Polo,Spain,Women
741,United States,Water Polo,United States of America,Men


In [16]:
# Calculate the counts of Men and Women in the 'Event' column
gender_counts = teams_df['Event'].value_counts().reset_index()
gender_counts.columns = ['Gender', 'Count']

# Filter to include only 'Men' and 'Women'
gender_counts_filtered = gender_counts[gender_counts['Gender'].isin(['Men', 'Women'])].copy()


# Calculate the total count for Men and Women
total_men_women = gender_counts_filtered['Count'].sum()

# Calculate the percentage of participation
gender_counts_filtered.loc[:, 'Percentage'] = (gender_counts_filtered['Count'] / total_men_women) * 100

display(gender_counts_filtered)

,Gender,Count,Percentage
0,Men,120,51.282051
1,Women,114,48.717949


In [20]:
# Count the number of unique NOCs for each Discipline in teams_df
country_counts_per_discipline = teams_df.groupby('Discipline')['NOC'].nunique().reset_index()
country_counts_per_discipline.columns = ['Discipline', 'Number of Participating Countries']

# Sort the results to see which disciplines have the most participating countries
country_counts_per_discipline_sorted = country_counts_per_discipline.sort_values(by='Number of Participating Countries', ascending=False)

display(country_counts_per_discipline_sorted.head())

,Discipline,Number of Participating Countries
4,Athletics,33
1,Archery,31
15,Swimming,30
16,Table Tennis,25
10,Football,24


### Insight 1: Top 10 Countries by Total Medals

**Question:** Which countries won the most medals in total?

In [21]:
# Sort medals_df by 'Total' medals in descending order
top_10_countries_medals = medals_df.sort_values(by='Total', ascending=False).head(10)

display(top_10_countries_medals)

,Rank,Team/NOC,Gold,Silver,Bronze,Total,Rank by Total
0,1,United States of America,39,41,33,113,1
1,2,People's Republic of China,38,32,18,88,2
4,5,ROC,20,28,23,71,3
3,4,Great Britain,22,21,22,65,4
2,3,Japan,27,14,17,58,5
5,6,Australia,17,7,22,46,6
9,10,Italy,10,10,20,40,7
8,9,Germany,10,11,16,37,8
6,7,Netherlands,10,12,14,36,9
7,8,France,10,12,11,33,10


**Explanation:** This table shows the top 10 countries that won the highest number of total medals. The United States of America is at the top with 113 medals.

### Insight 2: Medal Distribution of the Top Country

**Question:** How are the medals distributed (Gold, Silver, Bronze) for the country with the most total medals?

In [22]:
# Get the data for the top country (United States of America)
usa_medals = medals_df[medals_df['Team/NOC'] == 'United States of America']

display(usa_medals)

,Rank,Team/NOC,Gold,Silver,Bronze,Total,Rank by Total
0,1,United States of America,39,41,33,113,1


**Explanation:** This shows the breakdown of Gold, Silver, and Bronze medals won by the United States of America, the country with the highest total medal count.

### Insight 3: Countries with the Highest Success Ratio (Medals per Athlete)

**Question:** Which countries had the highest success ratio (total medals per athlete)?

In [23]:
# Sort the medals_with_athletes DataFrame by 'Success Ratio' in descending order
top_success_ratio_countries = medals_with_athletes.sort_values(by='Success Ratio', ascending=False).head(10)

display(top_success_ratio_countries)

,Rank,Team/NOC,Gold,Silver,Bronze,Total,Rank by Total,Number of Athletes,Success Ratio
72,72,San Marino,0,1,2,3,60,4,75.000000
62,63,Bermuda,1,0,0,1,77,2,50.000000
68,69,Armenia,0,2,2,4,47,16,25.000000
32,33,Georgia,2,5,1,8,29,35,22.857143
4,5,ROC,20,28,23,71,3,318,22.327044
49,50,Philippines,1,2,1,4,47,18,22.222222
1,2,People's Republic of China,38,32,18,88,2,401,21.945137
13,14,Cuba,7,3,5,15,18,69,21.739130
40,41,Qatar,2,0,1,3,60,14,21.428571
42,42,Kosovo,2,0,0,2,66,10,20.000000


**Explanation:** This table shows the top 10 countries based on their 'Success Ratio', which is the total number of medals won divided by the number of athletes from that country. A higher ratio suggests more medals were won relative to the size of their athlete delegation.

### Insight 4: Gender Participation in Teams

**Question:** What is the percentage distribution of Men's and Women's teams in the dataset?

In [24]:
# Calculate the counts of Men and Women in the 'Event' column from teams_df
gender_counts = teams_df['Event'].value_counts().reset_index()
gender_counts.columns = ['Gender', 'Count']

# Filter to include only 'Men' and 'Women'
gender_counts_filtered = gender_counts[gender_counts['Gender'].isin(['Men', 'Women'])].copy()

# Calculate the total count for Men and Women
total_men_women = gender_counts_filtered['Count'].sum()

# Calculate the percentage of participation
gender_counts_filtered.loc[:, 'Percentage'] = (gender_counts_filtered['Count'] / total_men_women) * 100

display(gender_counts_filtered)

,Gender,Count,Percentage
0,Men,120,51.282051
1,Women,114,48.717949


**Explanation:** This shows the percentage of teams that are categorized as 'Men' or 'Women' in the `teams_df`, giving us an idea of gender representation in team events.

### Insight 5: Disciplines with the Most Participating Countries

**Question:** Which disciplines have the highest number of participating countries?

In [25]:
# Display the sorted list of disciplines by the number of participating countries (calculated earlier)
display(country_counts_per_discipline_sorted.head(10))

,Discipline,Number of Participating Countries
4,Athletics,33
1,Archery,31
15,Swimming,30
16,Table Tennis,25
10,Football,24
8,Cycling Track,23
7,Beach Volleyball,23
3,Artistic Swimming,22
17,Triathlon,18
11,Handball,18


**Explanation:** This table lists the top 10 disciplines based on the number of unique countries that have teams participating in them, suggesting a wider global reach and potentially higher competition in these sports based on this metric.

### Insight 6: Total Number of Athletes from Top Medal-Winning Countries

**Question:** What is the total number of athletes from the top 10 medal-winning countries?

In [32]:
# Merge the top 10 countries by medals with the athlete counts
top_10_countries_athlete_counts = pd.merge(top_10_countries_medals, athlete_counts, left_on='Team/NOC', right_on='NOC', how='left')

display(top_10_countries_athlete_counts[['Team/NOC', 'Total', 'Number of Athletes']])

,Team/NOC,Total,Number of Athletes
0,United States of America,113,615
1,People's Republic of China,88,401
2,ROC,71,318
3,Great Britain,65,366
4,Japan,58,586
5,Australia,46,470
6,Italy,40,356
7,Germany,37,400
8,Netherlands,36,274
9,France,33,377


**Explanation:** This shows the total number of athletes that represented each of the top 10 medal-winning countries.

### Insight 7: Top Disciplines by Female and Male Entries

**Question:** Which disciplines have the highest number of female and male entries?

In [27]:
# Sort entries_gender_df by 'Female' in descending order and display top 5
print("Top 5 Disciplines by Female Entries:")
display(entries_gender_df.sort_values(by='Female', ascending=False).head())

# Sort entries_gender_df by 'Male' in descending order and display top 5
print("\nTop 5 Disciplines by Male Entries:")
display(entries_gender_df.sort_values(by='Male', ascending=False).head())

Top 5 Disciplines by Female Entries:


,Discipline,Female,Male,Total
4,Athletics,969,1072,2041
36,Swimming,361,418,779
20,Football,264,344,608
29,Rowing,257,265,522
23,Hockey,192,192,384



Top 5 Disciplines by Male Entries:


,Discipline,Female,Male,Total
4,Athletics,969,1072,2041
36,Swimming,361,418,779
20,Football,264,344,608
29,Rowing,257,265,522
24,Judo,192,201,393


**Explanation:** These tables show the disciplines with the highest number of female and male participants, respectively, based on the entry numbers.

### Insight 8: Disciplines with Significant Gender Disparity in Entries

**Question:** Which disciplines have a significant difference in the number of male and female entries?

In [28]:
# Calculate the absolute difference between male and female entries
entries_gender_df['Gender Difference'] = abs(entries_gender_df['Male'] - entries_gender_df['Female'])

# Sort by the absolute difference in descending order
gender_disparity_disciplines = entries_gender_df.sort_values(by='Gender Difference', ascending=False).head(10)

display(gender_disparity_disciplines[['Discipline', 'Female', 'Male', 'Gender Difference']])

,Discipline,Female,Male,Gender Difference
3,Artistic Swimming,105,0,105
4,Athletics,969,1072,103
45,Wrestling,96,193,97
28,Rhythmic Gymnastics,96,0,96
9,Boxing,102,187,85
20,Football,264,344,80
15,Cycling Road,70,131,61
36,Swimming,361,418,57
6,Baseball/Softball,90,144,54
18,Equestrian,73,125,52


**Explanation:** This table highlights disciplines where there is a large difference between the number of male and female entries, indicating potential gender disparity in participation within those sports based on the entry data.

### Insight 9: Total Number of Entries Across All Disciplines

**Question:** What is the total number of entries across all disciplines?

In [29]:
# Sum the 'Total' column in the entries_gender_df
total_entries = entries_gender_df['Total'].sum()

print(f"Total number of entries across all disciplines: {total_entries}")

Total number of entries across all disciplines: 11316


**Explanation:** This gives us the overall scale of participation in the Olympics based on the total number of entries recorded in the `entries_gender_df`.

### Insight 10: Disciplines with Zero Female or Male Entries

**Question:** Which disciplines have entries for only one gender?

In [30]:
# Filter for disciplines with zero female entries
female_only_disciplines = entries_gender_df[entries_gender_df['Female'] == 0]

print("Disciplines with Zero Female Entries (Male Only or Mixed):")
display(female_only_disciplines[['Discipline', 'Female', 'Male', 'Total']])

# Filter for disciplines with zero male entries
male_only_disciplines = entries_gender_df[entries_gender_df['Male'] == 0]

print("\nDisciplines with Zero Male Entries (Female Only or Mixed):")
display(male_only_disciplines[['Discipline', 'Female', 'Male', 'Total']])

Disciplines with Zero Female Entries (Male Only or Mixed):


,Discipline,Female,Male,Total



Disciplines with Zero Male Entries (Female Only or Mixed):


,Discipline,Female,Male,Total
3,Artistic Swimming,105,0,105
28,Rhythmic Gymnastics,96,0,96


**Explanation:** These tables show disciplines where the entry data indicates participation from only one gender, or potentially includes mixed-gender events where one gender has zero dedicated entries.